# Scraping Chipotle Press Release

Install Requirements and Import Libraries

In [1]:
import pandas as pd

Parse the html code using BeautifulSoup

In [2]:
from bs4 import BeautifulSoup

with open(r'C:\Users\achal\OneDrive\Desktop\Quilt\chipotle.html', encoding='utf8') as f:
    soup = BeautifulSoup(f, 'html.parser')
    print(soup.prettify())

<document>
 <type>
  EX-99.1
  <sequence>
   2
   <filename>
    cmg-20230207xex99_1.htm
    <description>
     EX-99.1
     <text>
      <!--HTML document created with Certent Disclosure Management 22.4.0.1-->
      <!--Created on: 2/7/2023 12:43:48 PM-->
      <html>
       <head>
        <title>
         Ex 99.1 Q42022
        </title>
        <script>
         bazadebezolkohpepadr="1684781304"
        </script>
        <script defer="" src="https://www.sec.gov/akam/13/646bbaef" type="text/javascript">
        </script>
       </head>
       <body>
        <div style="margin-left:36pt;margin-right:36pt;">
         <p style="margin:4.5pt 0pt 9pt;font-family:Times New Roman;font-size: 8pt">
          <font style="display: inline;font-family:Calibri;font-size:8pt;">
           ﻿
          </font>
         </p>
         <p style="margin:0pt;text-align:right;font-family:Times New Roman;font-size: 10pt">
          <font style="display: inline;font-family:Calibri;font-weight:bold;color:#00

Look at any pattern in the webpage and scrap text and tables based on that pattern

In this case, I see the pattern of text and tables divided using hr_tag as below

In [3]:
# Find the occurrence of the specified 'hr' tag
hr_tag = soup.find('hr', {'align': 'center', 'size': '3', 'style': 'color:#999999', 'width': '100%'})


Get any text before first hr_tag and save it as txt file

In [4]:

# Get all text elements before the 'hr' tag
text_elements = []

for elem in hr_tag.previous_elements:
    if isinstance(elem, str):
        text_elements.append(elem.strip())

# Reverse the list of text elements to maintain the correct order
text_elements = text_elements[::-1]

# Join the list to get the final text
text_before_hr = ' '.join(text_elements).strip()

print(text_before_hr)

with open('extracted_text.txt', 'a', encoding='utf-8') as f:
    # Write the extracted text to the file
    f.write(text_before_hr)

EX-99.1 2 cmg-20230207xex99_1.htm EX-99.1  HTML document created with Certent Disclosure Management 22.4.0.1  Created on: 2/7/2023 12:43:48 PM    Ex 99.1 Q42022  bazadebezolkohpepadr="1684781304"    ﻿    Exhibit 99. 1       ﻿               EARNINGS RELEASE       PR Contact: Laurie Schalow       (949) 524-4035       MediaRelations@chipotle.com     IR Contact: Cindy Olsen , CFA   (949) 524-4 205   Cindy.Olsen@chipotle.com      ﻿    Chipotle ANNOUNCES FOURTH QUARTER AND FULL YEAR 20 2 2 RESULTS    FY22 Operating income increases 44.2% and comparable restaurant sales increase 8.0% as margins expand     ﻿    NEWPORT BEACH, Calf. – February 7 , 202 3 – Chipotle Mexican Grill, Inc. (NYSE: CMG) today reported financial results for its fourth quarter and fiscal year ended December 31, 202 2 .    Fourth quarter highlights, year over year:     ·     Total r evenue increased 11.2% to $2.2 billion    ·     Comparable restaurant sales increased 5.6%    ·     In-restaurant sales increased 17.5 %, whi

Get all hr_tags

In [5]:
hr_tags = soup.find_all('hr', {'align': 'center', 'size': '3', 'style': 'color:#999999', 'width': '100%'})

Get texts between hr_tags and save it as txt files. 

In [6]:

# Iterate through pairs of hr tags
for i in range(0,3):
    start_tag = hr_tags[i]
    end_tag = hr_tags[i + 1]

    # Extract the content between the hr tags
    content = []
    for elem in start_tag.find_next_siblings():
        if elem == end_tag:
            break
        content.append(elem)

    # Create a new BeautifulSoup object with the extracted content
    content_soup = BeautifulSoup(''.join(str(item) for item in content), 'html.parser')

    # Extract the text
    text = content_soup.get_text(strip=True)
    
    # Do something with the extracted text
    print(text)
    filename = f"extracted_text_{i+1}.txt"
    with open(filename, 'a', encoding='utf-8') as f:
        # Write the title to the file
        f.write(text)
        

"We delivered strong growth in 2022, expanding average unit volumes and restaurant level margin, while openingthe highestnumberof new restaurantsin six years, despite facing a challenging and fluid macro environment," said Brian Niccol, Chairman and CEO, Chipotle. "Our continued focus on recruiting and retaining the best people, delivering Chipotle’s operational standards with delicious food prepared fresh daily uniquely positions Chipotle to successfully expand to 7,000 restaurants over the long term.”Results for the three months ended December 31, 2022:Total revenue in the fourth quarter was$2.2billion, an increase of11.2%compared to the fourth quarter of 2021. The increase in total revenue was driven by a5.6%increase in comparable restaurant sales and new restaurant openings.Our in-restaurant sales increased17.5% in the three months ended December 31, 2022, as compared to the three months ended December 31, 2021, while digital sales represented37.4% of total food and beverage revenu

Get the titles and its associated tables between hr_tags with the best structure possible and save it as txt files.

In [7]:

# Iterate through pairs of hr tags
for i in range(3,len(hr_tags) - 1):
    start_tag = hr_tags[i]
    end_tag = hr_tags[i + 1]

    # Extract the content between the hr tags
    content = []
    for elem in start_tag.find_next_siblings():
        if elem == end_tag:
            break
        content.append(elem)

    # Create a new BeautifulSoup object with the extracted content
    content_soup = BeautifulSoup(''.join(str(item) for item in content), 'html.parser')

    # Extract the title
    title_parts = [font.get_text(strip=True) for font in content_soup.find_all('font', style="display: inline;font-weight:bold;")]
    title = ' '.join(title_parts)

    tables = content_soup.find_all('table')

    # Loop through all tables and extract their text content
    for table in tables:
        # Create an empty list to store table rows
        rows = []
        # Loop through all table rows and extract their cells
        for tr in table.find_all('tr'):
            cells = []
            # Loop through all cells and append their text content to cells list
            for td in tr.find_all(['td', 'th']):
                cells.append(td.get_text(strip=True))
            # Append the cells list to rows list
            rows.append(cells)
        
        # Print the title
        print(title)
        
        # Print the table as text
        print('\n'.join(['\t'.join(row) for row in rows]))
        
        filename = f"extracted_table_{i+1}.txt"
        with open(filename, 'a', encoding='utf-8') as f:
            # Write the title to the file
            f.write(title + '\n')
            # Write the table as text to the file
            f.write('\n'.join(['\t'.join(row) for row in rows]) + '\n')


C HIPOTLE MEXICAN GRILL, INC. C ONSOLIDATED STATEMENTS OF INCOME (in thousands, except per share data) (unaudited) ﻿ ﻿
﻿											
﻿	Three months ended December 31,
﻿	2022		2021
Food and beverage revenue	$	2,163,907		99.2	%		$	1,939,405		98.9	%
Delivery service revenue		16,692		0.8				21,228		1.1	
Total revenue		2,180,599		100.0				1,960,633		100.0	
Restaurant operating costs (exclusive of depreciation and amortization shown separately below):											
Food, beverage and packaging		638,851		29.3				620,150		31.6	
Labor		558,914		25.6				516,829		26.4	
Occupancy		118,648		5.4				107,184		5.5	
Other operating costs		341,644		15.7				320,452		16.3	
General and administrative expenses		135,073		6.2				159,777		8.1	
Depreciation and amortization		74,012		3.4				66,262		3.4	
Pre-opening costs		11,341		0.5				6,984		0.4	
Impairment, closure costs, and asset disposals		5,785		0.3				4,699		0.2	
Total operating expenses		1,884,268		86.4				1,802,337		91.9	
Income from operations		296,331		13

For every 2 hr_tags there is one txt file. Next step is to use these txt files for preprocessing to furthur do the modelling.

Please refer to chipotle_model.ipynb for next steps